- 01 고객 3,500명에 대한 학습용 데이터(x_train.csv,y_train.csv)를 이용하여 성별예측모형을 만든 후,
- 이를 평가용 데이터(x_test.csv)에 적용하여 얻은 2482명의 고객 성별 예측값(남자일 확률)을 아래 형식의 csv파일로 제출하시오.
- (제출할 모델의 성능은 ROC_AUC 평가지표에 따라 채점)
- 제출형태
- 변수명.to_csv('수험번호.csv',index=False)

In [109]:
import pandas as pd
x_train = pd.read_csv('data/x_train.csv', encoding='cp949')
x_test = pd.read_csv('data/x_test.csv', encoding='cp949')
y_train = pd.read_csv('data/y_train.csv', encoding='cp949')

# 1. 라이브러리 및 데이터 확인
import pandas as pd
import numpy as np
# print(x_train.shape)
# print(x_test.shape)
# print(y_train.shape)
# print(x_train.info())
# print(x_test.info())
# print(y_train.info()) # 환불금액 결측치
# object type
cust_id = x_test['cust_id'].copy()
x_train = x_train.drop(columns=['cust_id'])
x_test = x_test.drop(columns=['cust_id'])
y_train = y_train.drop(['cust_id'], axis=1)
# print(x_train.info())
# print(x_test.info())
# print(y_train.info())
# 2. 데이터 탐색(EDA)
# print(x_train.describe().T)
# print(x_test.describe().T)
# print(x_train.describe(include='object').T)
# print(x_test.describe(include='object').T)
# print(y_train.value_counts())
# 3. 데이터 전처리 및 분리
# 1)결측치 2)이상치 3)변수 처리
# print(x_train.isnull().sum())
# print(x_test.isnull().sum())
# print(y_train.isnull().sum())
med = x_train['환불금액'].median()
x_train['환불금액'] = x_train['환불금액'].fillna(med)
x_test['환불금액'] = x_test['환불금액'].fillna(med)
# print(x_train.isnull().sum())
# print(x_test.isnull().sum())
# print(y_train.isnull().sum())
x_train = pd.get_dummies(x_train, dtype='uint8')
x_test = pd.get_dummies(x_test, dtype='uint8')
# print(x_train.info())
# print(x_test.info())
# print(x_train.shape)
# print(x_test.shape)
x_test = x_test.reindex(columns=x_train.columns, fill_value=0)
# print(x_train.shape)
# print(x_test.shape)
# print(x_train.head(3))
# print(x_test.head(3))
# 데이터 분리
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train,
                                                  y_train['gender'],
                                                  test_size=0.2,
                                                  stratify=y_train['gender']
                                                  )
# print(x_train.shape)
# print(x_val.shape)
# print(y_train.shape)
# print(y_val.shape)

# 4. 모델링 및 성능평가
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
model = RandomForestClassifier(n_estimators=50, random_state=0)
model.fit(x_train, y_train)
y_pred = model.predict(x_val)
#print(y_pred)

from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, precision_score, confusion_matrix
acc = accuracy_score(y_val, y_pred)
auc = roc_auc_score(y_val, y_pred)
cm = confusion_matrix(y_val, y_pred)
print(acc)
print(auc)
print(cm)

# y_result = model.predict(x_test)
# y_result_prob = model.predict_proba(x_test)
# # print(y_result[:5])
# # print(y_result_prob[:5])
# result = pd.DataFrame({'result':y_result,
#                        'prob0':y_result_prob[:,0],
#                        'prob1':y_result_prob[:,1]})
# # print(result[:5])

# # 5. 예측값 제출
# pd.DataFrame({'cust_id':cust_id, 'gender':y_result_prob[:,0]}).to_csv('단축키2유형1.csv', index=False)
# df = pd.read_csv('단축키2유형1.csv')
# print(df.head(3))

0.6214285714285714
0.5650825277775361
[[346  91]
 [174  89]]


In [97]:
# preprocessing
import pandas as pd
x_train = pd.read_csv('data/x_train.csv', encoding='cp949')
x_test = pd.read_csv('data/x_test.csv', encoding='cp949')
y_train = pd.read_csv('data/y_train.csv', encoding='cp949')
# print(x_train.head(3))
# print(x_test.head(3))
# print(y_train.head(3))
# print(x_train.info())
# print(x_test.info())
# print(y_train.info())
cust_id = x_test.copy()
x_train = x_train.drop(['cust_id'], axis=1)
y_train = y_train.drop(columns=['cust_id'])
x_test = x_test.drop(columns=['cust_id'])
# print(x_train.isnull().sum())
# print(x_test.isna().sum())

# 환불금액이 결측치라는 건 환불금액이 없다는 말로 파악
x_train['환불금액'] = x_train['환불금액'].fillna(0)
x_test['환불금액'] = x_test['환불금액'].fillna(0)
# print(x_train.isnull().sum())
# print(x_test.isna().sum())

# 인코딩 변환
# print(x_train['주구매상품'].unique())
# print(x_test['주구매상품'].unique())
# print(x_train['주구매상품'].unique().size)
# print(x_test['주구매상품'].unique().size)

from sklearn.preprocessing import LabelEncoder
encoder_상품 = LabelEncoder()
encoder_지점 = LabelEncoder()
x_train['주구매상품'] = encoder_상품.fit_transform(x_train['주구매상품'])
x_train['주구매지점'] = encoder_지점.fit_transform(x_train['주구매지점'])
#print(x_train.head(3))
def safe_transform(encoder, labels):
    unseen_label = -1
    label_mapping = {label:idx for idx,label in enumerate(encoder.classes_)}
    return np.array([label_mapping.get(label, unseen_label) for label in labels])

x_test['주구매상품'] = safe_transform(encoder_상품, x_test['주구매상품'])
x_test['주구매지점'] = safe_transform(encoder_지점, x_test['주구매지점'])
#print(x_test.head(3))
# print(x_train.info())
# print(x_test.info())
# 표준화를 할 필요가 있나?
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = pd.DataFrame(scaler.fit_transform(x_train), columns=x_train.columns)
# print(x_train.head(3))
# print(x_train.describe().T)
x_test = pd.DataFrame(scaler.transform(x_test), columns=x_test.columns)
# print(x_test.head(3))
# print(x_test.describe().T)

# from sklearn.model_selection import train_test_split
# x_train, x_val, y_train, y_val = train_test_split(x_train,
#                                                   y_train['gender'],
#                                                   test_size=0.2,
#                                                   stratify=y_train['gender'],
#                                                   random_state=24
#                                                   )
# # print(x_train.shape)
# # print(x_val.shape)
# # print(y_train.shape)
# # print(y_val.shape)
# from sklearn.ensemble import RandomForestClassifier
# model = RandomForestClassifier()
# model.fit(x_train, y_train)
# y_pred = model.predict(x_val)

# from sklearn.metrics import accuracy_score, roc_auc_score
# acc = accuracy_score(y_val, y_pred)
# auc = roc_auc_score(y_val, y_pred)
# # print(acc)
# # print(auc)

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(x_train, y_train['gender'])
y_pred = model.predict(x_train)
#print(y_result)

from sklearn.metrics import accuracy_score, roc_auc_score
acc2 = accuracy_score(y_train['gender'], y_pred)
auc2 = roc_auc_score(y_train.values.ravel(), y_pred)
# print(acc2)
# print(auc2)

y_result = model.predict(x_test)
#print(y_result)
y_result_prob = model.predict_proba(x_test)
print(y_result_prob)

[[0.45 0.55]
 [0.82 0.18]
 [0.87 0.13]
 ...
 [0.45 0.55]
 [0.61 0.39]
 [0.49 0.51]]


In [98]:
# GridSearchCV
import pandas as pd
x_train = pd.read_csv('data/x_train.csv', encoding='cp949')
x_test = pd.read_csv('data/x_test.csv', encoding='cp949')
y_train = pd.read_csv('data/y_train.csv', encoding='cp949')
# print(x_train.head(3))
# print(x_test.head(3))
# print(y_train.head(3))
# print(x_train.info())
# print(x_test.info())
# print(y_train.info())
cust_id = x_test.copy()
x_train = x_train.drop(['cust_id'], axis=1)
y_train = y_train.drop(columns=['cust_id'])
x_test = x_test.drop(columns=['cust_id'])
# print(x_train.isnull().sum())
# print(x_test.isna().sum())

# 환불금액이 결측치라는 건 환불금액이 없다는 말로 파악
x_train['환불금액'] = x_train['환불금액'].fillna(0)
x_test['환불금액'] = x_test['환불금액'].fillna(0)
# print(x_train.isnull().sum())
# print(x_test.isna().sum())

# 인코딩 변환
# print(x_train['주구매상품'].unique())
# print(x_test['주구매상품'].unique())
# print(x_train['주구매상품'].unique().size)
# print(x_test['주구매상품'].unique().size)

from sklearn.preprocessing import LabelEncoder
encoder_상품 = LabelEncoder()
encoder_지점 = LabelEncoder()
x_train['주구매상품'] = encoder_상품.fit_transform(x_train['주구매상품'])
x_train['주구매지점'] = encoder_지점.fit_transform(x_train['주구매지점'])
#print(x_train.head(3))
def safe_transform(encoder, labels):
    unseen_label = -1
    label_mapping = {label:idx for idx,label in enumerate(encoder.classes_)}
    return np.array([label_mapping.get(label, unseen_label) for label in labels])

x_test['주구매상품'] = safe_transform(encoder_상품, x_test['주구매상품'])
x_test['주구매지점'] = safe_transform(encoder_지점, x_test['주구매지점'])
#print(x_test.head(3))
# print(x_train.info())
# print(x_test.info())
# 표준화를 할 필요가 있나?
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = pd.DataFrame(scaler.fit_transform(x_train), columns=x_train.columns)
# print(x_train.head(3))
# print(x_train.describe().T)
x_test = pd.DataFrame(scaler.transform(x_test), columns=x_test.columns)
# print(x_test.head(3))
# print(x_test.describe().T)

# grid search
from sklearn.model_selection import GridSearchCV
params = {'max_depth':[3,4,5,6],
          'min_samples_leaf':[3,4,5,6],
          'min_samples_split':[2,4,6]}
rf = RandomForestClassifier(n_estimators=50, random_state=0)
grid_cv = GridSearchCV(rf, param_grid=params, cv=2)
grid_cv.fit(x_train, y_train['gender'])

# print("최적 하이퍼파라미터 : ", grid_cv.best_params_)
# print("최적 score : ", grid_cv.best_score_)

model_cv = RandomForestClassifier(n_estimators=50, 
                                  max_depth=5, 
                                  min_samples_leaf=4,
                                  min_samples_split=2,
                                  random_state=0)
model_cv.fit(x_train, y_train['gender'])
y_result = model_cv.predict(x_test)
print(y_result[:5])

[0 0 0 0 1]


In [108]:
# KNN
import pandas as pd
x_train = pd.read_csv('data/x_train.csv', encoding='cp949')
x_test = pd.read_csv('data/x_test.csv', encoding='cp949')
y_train = pd.read_csv('data/y_train.csv', encoding='cp949')
# print(x_train.head(3))
# print(x_test.head(3))
# print(y_train.head(3))
# print(x_train.info())
# print(x_test.info())
# print(y_train.info())
cust_id = x_test.copy()
x_train = x_train.drop(['cust_id'], axis=1)
y_train = y_train.drop(columns=['cust_id'])
x_test = x_test.drop(columns=['cust_id'])
# print(x_train.isnull().sum())
# print(x_test.isna().sum())

# 환불금액이 결측치라는 건 환불금액이 없다는 말로 파악
x_train['환불금액'] = x_train['환불금액'].fillna(0)
x_test['환불금액'] = x_test['환불금액'].fillna(0)
# print(x_train.isnull().sum())
# print(x_test.isna().sum())

# 인코딩 변환
# print(x_train['주구매상품'].unique())
# print(x_test['주구매상품'].unique())
# print(x_train['주구매상품'].unique().size)
# print(x_test['주구매상품'].unique().size)

from sklearn.preprocessing import LabelEncoder
encoder_상품 = LabelEncoder()
encoder_지점 = LabelEncoder()
x_train['주구매상품'] = encoder_상품.fit_transform(x_train['주구매상품'])
x_train['주구매지점'] = encoder_지점.fit_transform(x_train['주구매지점'])
#print(x_train.head(3))
def safe_transform(encoder, labels):
    unseen_label = -1
    label_mapping = {label:idx for idx,label in enumerate(encoder.classes_)}
    return np.array([label_mapping.get(label, unseen_label) for label in labels])

x_test['주구매상품'] = safe_transform(encoder_상품, x_test['주구매상품'])
x_test['주구매지점'] = safe_transform(encoder_지점, x_test['주구매지점'])
#print(x_test.head(3))
# print(x_train.info())
# print(x_test.info())
# 표준화를 할 필요가 있나?
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = pd.DataFrame(scaler.fit_transform(x_train), columns=x_train.columns)
# print(x_train.head(3))
# print(x_train.describe().T)
x_test = pd.DataFrame(scaler.transform(x_test), columns=x_test.columns)
# print(x_test.head(3))
# print(x_test.describe().T)

from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train,
                                                  y_train['gender'],
                                                  test_size=0.2,
                                                  stratify=y_train['gender'],
                                                  random_state=24
                                                  )
# print(x_train.shape)
# print(x_val.shape)
# print(y_train.shape)
# print(y_val.shape)

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
knn.fit(x_train, y_train)
y_pred = knn.predict(x_val)
y_pred_prob = knn.predict_proba(x_val)
print(y_pred[:5])
# print(y_result_prob[:5])

from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, precision_score, confusion_matrix
acc = accuracy_score(y_val, y_pred)
auc = roc_auc_score(y_val, y_pred)
cm = confusion_matrix(y_val, y_pred)
print(acc)
print(auc)
#print(cm)

[0 0 0 1 1]
0.6257142857142857
0.585925468324473


- 02 보험관련 학습용 데이터 Insurance_train_10.csv 데이터를 이용하여 "Segmentation"예측모형을 개발하려고 한다.
- 자동차 회사는 기존 시장에서 영업팀은 모든 고객을 4개의 "Segmentation"으로 분류하고 새로운 시장에서 동일한 전략을 사용할 계획이다.
- 채점은 제출된 CSV 파일의 macro-F1 평가지표에 따라 구간별로 배정 과적합 등의 사유로 평가용 데이터의 평가지표와 채점용 데이터의 평가지표는 다를 수 있음

In [142]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
train = pd.read_csv('data/Insurance_train_10.csv')
test = pd.read_csv('data/Insurance_test_10.csv')
x_train = train.copy()
x_test = test.copy()

# 1. 라이브러리 및 데이터 확인
# print(x_train.head(3))
# print(x_test.head(3))
# print(y_train.head(3))
# print(x_train.info())
# print(x_test.info())
# print(y_train.info()) # 결측치 ever_married, graduated, profession

# 2. 데이터 탐색(EDA)
# print(x_train.describe().T)
# print(x_test.describe().T)
# print(x_train.info()) # dtype object
# print(x_train.describe(include='object').T)
# print(x_test.describe(include='object').T) # ever_married 4056개 (Yes) / graduated 4370개 (Yes) 
#                                            # / profession 2247개 (Artist)
#print(y_train.value_counts())
#print(x_train['Profession'].value_counts())
# print(x_train.isnull().sum())
# print(x_test.isnull().sum())
# print(y_train.isnull().sum())

# 3. 데이터 전처리 및 분리
x_train = x_train.dropna()
x_test = x_test.dropna()
y_train = x_train['Segmentation']
x_train = x_train.drop(['Segmentation'], axis=1)
# print(x_train.isnull().sum())
# print(x_test.isnull().sum())
# print(x_train['Gender'].unique())
# print(x_train['Ever_Married'].unique())
# print(x_train['Graduated'].unique())
# print(x_train['Profession'].unique())
#print(x_train['Spending_Score'].unique())

# print(x_test['Gender'].unique())
# print(x_test['Ever_Married'].unique())
# print(x_test['Graduated'].unique())
# print(x_test['Profession'].unique())
# print(x_test['Spending_Score'].unique())

x_train['Gender'] = x_train['Gender'].replace(['Male','Female'],[0,1])
x_train['Ever_Married'] = x_train['Ever_Married'].replace(['No','Yes'],[0,1])
x_train['Graduated'] = x_train['Graduated'].replace(['No','Yes'],[0,1])
x_train['Profession'] = x_train['Profession'].map({'Healthcare':0, 'Engineer':1, 
                                                   'Lawyer':2, 'Artist':3, 
                                                   'Doctor':4, 'Homemaker':5,
                                                   'Entertainment':6, 'Marketing':8, 'Executive':9})
x_train['Spending_Score'] = x_train['Spending_Score'].replace(['Low','High','Average'],[0,1,2])

x_test['Gender'] = x_test['Gender'].replace(['Male','Female'],[0,1])
x_test['Ever_Married'] = x_test['Ever_Married'].replace(['No','Yes'],[0,1])
x_test['Graduated'] = x_test['Graduated'].replace(['No','Yes'],[0,1])
x_test['Profession'] = x_test['Profession'].map({'Healthcare':0, 'Engineer':1, 
                                                   'Lawyer':2, 'Artist':3, 
                                                   'Doctor':4, 'Homemaker':5,
                                                   'Entertainment':6, 'Marketing':8, 'Executive':9})
x_test['Spending_Score'] = x_test['Spending_Score'].replace(['Low','High','Average'],[0,1,2])
# 표준화
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = pd.DataFrame(scaler.fit_transform(x_train), columns=x_train.columns)
x_test = pd.DataFrame(scaler.transform(x_test), columns=x_test.columns)
# print(x_train.head(3))
# print(x_test.head(3))

# from sklearn.model_selection import train_test_split
# x_train, x_val, y_train, y_val = train_test_split(x_train,
#                                                   y_train,
#                                                   test_size=0.2,
#                                                   stratify=y_train,
#                                                   )
# print(x_train.shape)
# print(x_val.shape)
# print(y_train.shape)
# print(y_val.shape)

# 4. 모델링 및 성능평가
# from sklearn.ensemble import RandomForestClassifier
# model = RandomForestClassifier()
# model.fit(x_train, y_train)

# y_pred = model.predict(x_val)

# from sklearn.metrics import f1_score, accuracy_score
# f1 = f1_score(y_val, y_pred, average='macro')
# acc = accuracy_score(y_val, y_pred)
# print(f1)
# print(acc)

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(x_train, y_train)
y_result = model.predict(x_train)

from sklearn.metrics import f1_score
f1 = f1_score(y_train, y_result, average='macro')
#print(f1)

y_submit = model.predict(x_test)
print(y_submit)

[2 3 3 ... 2 3 4]


- 03 워싱턴 D.C의 Capital Bikeshare 프로그램에서 자전거 대여 수요를 예측하기 위한 프로젝트이다.

In [174]:
# 1. 라이브러리 및 데이터 확인
import pandas as pd
df = pd.read_csv('data/bike_train.csv')
#print(df.head(3))
#print(df.shape)
#print(df.info()) # datetime dtype object
# 3. 데이터 전처리 및 분리
import datetime as dt
df['datetime'] = pd.to_datetime(df['datetime'])
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['day'] = df['datetime'].dt.day
df['hour'] = df['datetime'].dt.hour
#print(df.head(3))
drop_columns = ['datetime', 'casual', 'registered']
df = df.drop(drop_columns, axis=1)
#print(df.head(3))
#print(df.isnull().sum())
y = df['count']
#print(y.shape)
x = df.drop(['count'], axis=1)
#print(x.shape)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
x_train, x_val, y_train, y_val = train_test_split(x,
                                                  y,
                                                  test_size=0.3,
                                                  random_state=0)
# print(x_train.shape)
# print(x_val.shape)
# print(y_train.shape)
# print(y_val.shape)
# 4. 모델링 및 성능평가
reg = LinearRegression()
reg.fit(x_train, y_train)
y_pred = reg.predict(x_val)
#print(y_pred)
from sklearn.metrics import mean_absolute_error, mean_squared_error
#print(mean_absolute_error(y_val, y_pred))
rmse = mean_squared_error(y_val, y_pred, squared=False)
#print(rmse)
#print(mean_squared_error(y_val, y_pred, squared=True))
result_df = pd.DataFrame(y_val.values, columns=['real_count'])
import numpy as np
result_df['predict_count'] = np.round(y_pred)
result_df['diff'] = np.abs(result_df['real_count']-result_df['predict_count'])
#print(result_df.sort_values('diff', ascending=False).head(10))
after_df = pd.get_dummies(df, columns=['year','month','day','hour','holiday','workingday','season','weather'], dtype='uint8')
#print(after_df)
y = after_df['count']
x = after_df.drop(['count'], axis=1)

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
x_train2, x_val2, y_train2, y_val2 = train_test_split(x,
                                                      y,
                                                      test_size=0.3,
                                                      random_state=0)
reg2 = LinearRegression()
reg2.fit(x_train2, y_train2)
y_pred2 = reg2.predict(x_val2)

from sklearn.metrics import mean_squared_error, mean_absolute_error
# print(mean_absolute_error(y_val2, y_pred2))
# print(mean_squared_error(y_val2, y_pred2, squared=False))
from sklearn.ensemble import RandomForestRegressor
rf_reg = RandomForestRegressor(n_estimators=500)
rf_reg.fit(x_train2, y_train2)
y_pred2 = rf_reg.predict(x_val2)
#print(mean_absolute_error(y_val2, y_pred2))
print(mean_squared_error(y_val2, y_pred2, squared=False))

55.76036155726519
